# The word2vec Approach

-[Prashant Brahmbhatt](www.github.com/hashbanger)

____
[Reference](https://github.com/adventuresinML)

___________

In [0]:
import os
import math
import urllib
import random
import zipfile
import collections
import numpy as np
import datetime as dt
import tensorflow as tf  

import warnings
warnings.filterwarnings('ignore')

A straight-forward way of doing this would be to use a **one-hot** method of converting the word into a sparse representation with only one element of the vector set to 1, the rest being zero. 

For a sentence **''the cat sat on the mat''**, one hot enocding would be

$\begin{bmatrix}the\\cat\\sat\\on\\the\\mat\end{bmatrix}$ = $\begin{bmatrix}1&0&0&0&0\\0&1&0&0&0\\0&0&1&0&0\\0&0&0&1&0\\1&0&0&0&0\\0&0&0&0&1\end{bmatrix}$ 

But there are efficiency issues with the memory problems and the huge sparsity and also we lose word locality informtion.  
For instance, we might expect to see **“United”** and **“States”** to appear close together, or **“Soviet”** and **“Union”**.  Or **“food” and “eat”**, and so on. 

## Word2Vec Methodology

We require two goals to be fulfilled.  
- The first is the mapping of a high dimensional one-hot style representation of words to a lower dimensional vector. This might involve transforming a 10,000 columned matrix into a 300 columned matrix, for instance. This process is called word embedding.  
- The second goal is to do this while still maintaining word context and therefore, to some extent, meaning. 


There are two approaches for achieving these two goals:  
-  **Skip Gram Approach** - taking an input word and then attempting to estimate the probability of other words appearing close to that word.  
- **Continuous Bag Of Words** - it takes some context words as input and tries to find the single word that has the highest probability of fitting that context.


### Using Skip Gram Approach

 A **gram** is a group of **n words**, where **n** is the **gram window size**.  So for the sentence **“The cat sat on the mat”**, a 3-gram representation of this sentence would be **“The cat sat”, “cat sat on”, “sat on the”, “on the mat”.**
 

These grams are fed into the Word2Vec context prediction system. For instance, assume the input word is **“cat”** – the Word2Vec tries to predict the context (**“the”**, **“sat”**) from this supplied input word.  The Word2Vec system will move through all the supplied grams and input words and attempt to learn appropriate mapping vectors (embeddings) which produce high probabilities for the right context given the input words.  
   
The Word2Vec system is nothing but a neural network.

Assuming that **"the cat sat on the mat"** is a part of a larger database of 10k words vocb. We want to reduce this vocab to 300 words. If we take the word **“cat”** it will be one of the words in the 10,000 word vocabulary.  Therefore we can represent it as a 10,000 length one-hot vector.  We then interface this input vector to a 300 node hidden layer. The weights connecting this layer will be our **new word vectors***.   
The activations of the nodes in this hidden layer are simply linear summations of the weighted inputs i.e. no non-linear activation like **sig** or **tan** are applied.  
During training, we want to change the weights of this neural network so that words surrounding **“cat”** have a higher probability in the softmax output layer.

So, we will need to create a **10000 * 300** weight matrix connecting the inputs to 300 nodes.

The weight matrix essentially becomes a look-up or encoding table of our words.  Not only that, but these weight values contain context information due to the way we’ve trained our network.

Once we’ve trained the network, we abandon the softmax layer and just use the 10,000 x 300 weight matrix as our word embedding lookup table

### Preparing the Text Data

*Getting the data*

In [0]:
def maybe_download(filename, url, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urllib.request.urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        print(statinfo.st_size)
        raise Exception(
            'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

In [0]:
def collect_data(vocabulary_size=10000):
    url = 'http://mattmahoney.net/dc/'
    filename = maybe_download('text8.zip', url, 31344016)
    vocabulary = read_data(filename)
    print(vocabulary[:7])
    data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,
                                                                vocabulary_size)
    del vocabulary  # Hint to reduce memory.
    return data, count, dictionary, reverse_dictionary

*Extracting and Reading the data as strings*

In [0]:
def read_data(filename):
    #Extracting and reading as list of strings
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data
        

Since the stings in python 2 were not automatically unicode but were bytes rather.  
So, under the hood, **tensorflow.compat.as_str** converts both bytes and unicode strings to unicode strings.  
The **namelist()** function retrieves all the members of the archive – in this case there is only one member, so we access this using the zero index.

We have obtained the list of words

___________

Now we have to create our **skip-gram** batch. It includes the following steps.

- We will extract most common 10k words.
- collect all the unique words and index them with the unique index, to create an equivalent one-hot type input for the word. A dictionary is one way to accomplish this.  
- We will go through every word in the dataset and assign it to the unique integer word identified, as done in previous step. It will allow easy lookup of the word data stream

In [0]:
def build_dataset(words, n_words):
    """Process raw inputs into a dataset."""
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [9]:
vocabulary_size = 10000
data, count, dictionary, reverse_dictionary = collect_data(vocabulary_size=vocabulary_size)

Found and verified text8.zip
['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse']


Setted up a **“counter”** list, which will store the number of times a word is found within the data-set.  We are restricting our vocabulary to only 10,000 words, other words will be of **“UNK”** (Unknown) designation.  The initialized count list is then extended, using the Python collections module and the **Counter()** class and the associated **most_common()** function.

In [10]:
#it works as
collections.Counter('abracadabra').most_common(3)

[('a', 5), ('b', 2), ('r', 2)]

The value assigned to each unique word key is simply an increasing integer count of the size of the dictionary.  So, for instance, the most common word will receive the value 1, the second most common the value 2. Integer 0 is assigned to **UNK**

A list **data** is created of the same length as **words** but instead of being a list of individual words, it will instead be a list of integers – with each word now being represented by the unique integer that was assigned to this word in dictionary.  So, for the first sentence of our data-set   
**[‘anarchism’, ‘originated’, ‘as’, ‘a’, ‘term’, ‘of’, ‘abuse’]** --------------> **[5242, 3083, 12, 6, 195, 2, 3136]**.

**reverse_dictionary** that allows us to look up a word based on its unique integer identifier, rather than looking up the identifier based on the word i.e. the original dictionary.

____________

Now we create a data set comprising of our input words and associated grams

Following function will generate mini-batches to use during our training, which will consist of input words (stored in batch) and random associated context words within the gram as the labels to predict (stored in context).   

example: in the 5-gram **“the cat sat on the”**, the input word will be center word i.e. **“sat”** and the context words that will be predicted will be drawn randomly from the remaining words of the gram: **[‘the’, ‘cat’, ‘on’, ‘the’]**.  

In this function, the number of words drawn randomly from the surrounding context is defined by the argument **num_skips**.  

The size of the window of context words to draw from around the input word is defined in the argument **skip_window** – in the example above (**“the cat sat on the”**), we have a skip window width of 2 around the input word **"sat”**.

In [0]:
data_index = 0
def generate_batch(data, batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape = (batch_size), dtype = np.int32)
    context = np.ndarray(shape = (batch_size, 1), dtype = np.int32)
    span = 2 * skip_window + 1#[ skip_window input_word skip_window ]

#------- To create a buffer ---------------------------------------

    buffer = collections.deque(maxlen = span) #Deque is preferred over list 
                                              #where we need quicker append and pop operations
    for _ in range(span):
        buffer.append(data[data_index])
    data_index = (data_index + 1)% len(data)
    
    
#-------- To fill the batch and context variables-------------------

    for i in range(batch_size // num_skips):
        target = skip_window #input word at the centre
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span-1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window] #this is the input word
            context[i * num_skips + j, 0] = buffer[target] #these are the context words
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    # Backtracking to avoid skipping words in the end of a batch
    
    data_index = (data_index + len(data) - span) % len(data)
    return batch,context
            

First, the batch and label outputs are defined as variables of size batch_size.  
Then the span size is defined, which is basically the size of the word list that the input word and context samples will be drawn from.  Example, **“the cat sat on the”**span is  
**5 = 2 x skip window + 1**

This buffer will hold a maximum of **span** elements and will be a kind of moving window of words that samples are drawn from.  Whenever a new word index is added to the buffer, the left most element will drop out of the buffer to allow room for the new word index being added.  The position of the buffer in the input text stream is stored in a global variable **data_index** which is incremented each time a new word is added to the buffer.  If it gets to the end of the text stream, the **“% len(data)”** component of the index update will basically reset the count back to zero.

The first **target** word selected is the word at the center of the span of words and is therefore the input word.   
Then other words are randomly selected from the span of words, making sure that the input word is not selected as part of the context, and each context word is unique.  The batch variable will feature repeated input words (buffer[skip_window]) which are matched with each context word in context.

At the end of the function the batch and context are returned. Now we have a means of drawing batches of data from the data set.   
We now have to create code for word2vec in TF.   
We first have to create a validation set to measure our performance. We can do that by measuring the vectors closest together in vector-space, and make sure these words indeed are similar using our knowledge of English.

the code below shows how to grab some random validation words from the most common words in our vocabulary

In [0]:
# Picking a random validationset to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16
valid_window = 100
valid_examples = np.random.choice(valid_window, valid_size, replace = False)
num_sampled = 64 #Number of negative examples to sample

Above code randomly chooses 16 integers from 0-100 each corresponding to the integer indexes of the most common 100 words in our text data.  

## The Tensorflow Model

It consists of a NN with a word embedding matrix as the hidden layer, an output softmax layer in TF.  
We’ll be learning the best word embedding matrix and therefore we’ll be learning a reduced, context maintaining, mapping of words to vectors.

In [0]:
batch_size = 128
embedding_size = 128 # Dimension of embedding vactor
skip_window = 1      # Words to consider in left and right
num_skips = 2        # How many times to reuse an input to generate a context

#### Placeholders
We need to define some placeholders that will hold our input words and context words which we are trying to predict.  
We also need to create a constant to hold our validation set indexes.

In [0]:
train_inputs = tf.placeholder(tf.int32, shape = [batch_size])
train_context = tf.placeholder(tf.int32, shape = [batch_size, 1])
valid_dataset = tf.constant(valid_examples, dtype= tf.int32)

#### Setting up the embedding matrix tensor

First we create the embeddings variable, which is effectively the weights of the connections to the linear hidden layer.   
We initialize the variable with a random uniform distribution between -1.0 to 1.0.   
The size of this variable is **(vocabulary_size, embedding_size)** – the vocabulary_size is the 10,000 words that we have used to setup our data in the previous section.   
This is basically our one-hot vector input, where the only element with a value of “1” is the current input word, all the other values are set to “0”.   
The second dimension, **embedding_size**, is our hidden layer size, and is the length of our new, smaller, representation of our words.  We can also think of this tensor as a big lookup table – the rows are each word in our vocabulary, and the columns are our new vector representation of each of these words.

In [0]:
embeddings = tf.Variable(tf.random_uniform(shape= [vocabulary_size, embedding_size]
                                           , minval= -1.0, maxval = 1.0))
embed = tf.nn.embedding_lookup(embeddings, train_inputs)

  Here’s a simplified example (using dummy values), where vocabulary_size=7 and embedding_size=3:

### $$\begin{equation} 
\begin{array}{c|c c c} 
anarchism & 0.5 & 0.1 & -0.1\\ 
originated & -0.5 & 0.3 & 0.9 \\ 
as & 0.3 & -0.5 & -0.3 \\ 
a & 0.7 & 0.2 & -0.3\\ 
term & 0.8 & 0.1 & -0.1 \\ 
of & 0.4 & -0.6 & -0.1 \\ 
abuse & 0.7 & 0.1 & -0.4 
\end{array} 
\end{equation}$$

“anarchism” (which would actually be represented by a unique integer or one-hot vector) is now expressed as [0.5, 0.1, -0.1].  
We can “look up” anarchism by finding its integer index and searching the rows of embeddings to find the embedding vector: [0.5, 0.1, -0.1].

**tf.nn.embedding_lookup()** is a useful helper function in TensorFlow for this type of task.  
It takes an input vector of integer indexes – in this case our **train_input** tensor of training input words, and “looks up” these indexes in the supplied embeddings tensor.   
Therefore, this command will return the current embedding vector for each of the supplied input words in the training batch.  The full embedding tensor will be optimized during the training process.

#### Creating the weights and biases

In [0]:
# Weights connecting the hiddden and output layers
# (out_layer_size, hidden_layer_size) = (vocabulary_size, embedding_size)
weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size], stddev= 1.0 / math.sqrt(embedding_size)))

# single dimensional and the size of the output layer
biases = tf.Variable(tf.zeros([vocabulary_size]))

hidden_out = tf.matmul(embed, tf.transpose(weights)) + biases

#### Creating the softmax layer
To create a softmax operation and we will use cross entropy loss to optimize the weights, biases and embeddings of the model.  
We can use the **tf.nn.softmax_cross_entropy_with_logits** but to use this function we first have to convert the context words / integer indices into one-hot vectors.

In [0]:
train_one_hot = tf.one_hot(train_context, vocabulary_size )
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = hidden_out
                                                                          , labels = train_one_hot))
optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(cross_entropy)

#### The Cosine Similarity

To determine which words are similar to each other, we need to perform some sort of operation that measures the “distances” between the various word embedding vectors for the different words.  In this case, we will use the cosine similarity measure of distance between vectors.  

### $$similarity = cos(\theta) = \frac{\textbf{A}\cdot\textbf{B}}{\parallel\textbf{A}\parallel_2 \parallel \textbf{B} \parallel_2}$$

where the double-parallel lines denote the L2-norm.  
The best way to calculate the cosine similarity in TensorFlow is to normalize and then simply multiply each one.   
$$\frac{\textbf{A}}{\parallel\textbf{A}\parallel_2}
\frac{\textbf{B}}{\parallel\textbf{B}\parallel_2}$$


### $$ similarity = cos(\theta) = \textbf{A} . \textbf{B}$$

In [0]:
# Computing the cosine similarity between minibatch examples and all embeddings
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), axis = 1, keepdims= True))
normalized_embeddings = embeddings / norm 

We can look up our validation words / vectors using the **tf.nn.embedding_lookup()**

In [0]:
valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)

We are supplying a list of integers (that correspond to our validation vocabulary words) to the embedding_lookup() function, which looks up these rows in the normalized_embeddings tensor, and returns the subset of validation normalized embeddings.   
Now that we have the normalized validation tensor, valid_embeddings, we can multiply this by the full normalized vocabulary (normalized_embedding) to finalize our similarity calculation:

In [0]:
similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b= True)

This operation will return a **(validation_size, vocabulary_size)** sized tensor, where each row refers to one of our validation words and the columns refer to the similarity between the validation word and all the other words in the vocabulary.

## Running the Tensorflow Model

In [0]:
graph = tf.get_default_graph()

In [0]:
def run(graph, num_steps):
    with tf.Session(graph= graph) as session:
        # Initializing all the variables
        init = tf.initialize_all_variables()
        init.run()
        print('Initialized')

#----------------------------------------------------------------------------------------------

        #Now we initialize the variables and feed in each data batch to the training loop,
        # printing the average loss every 2000 iterations.

        average_loss = 0
        for step in range(num_steps):
            batch_inputs, batch_context = generate_batch(data, batch_size, num_skips, skip_window)
            feed_dict = {train_inputs: batch_inputs, train_context: batch_context}
            
            # We perform one update step by evaluating the optimizer op (including it
            # in the list of returned values for session.run()
            _, loss_val = session.run([optimizer, cross_entropy], feed_dict = feed_dict)
            average_loss = average_loss + loss_val
            
            if step % 2000 == 0:
                if step > 0:
                    average_loss /= 2000
                # The average loss is an estimate of the loss over the last 2000 batches.
                print('Average loss at step {} is {}'.format(step, average_loss))
                average_loss = 0

#--------------------------------------------------------------------------------------------------
            #This function first evaluates the similarity operation, which returns an array of cosine similarity 
            # values for each of the validation words.  Then we iterate through each of the validation words,
            # taking the top 8 closest words by using argsort() on the negative of the similarity to 
            #arrange the values in descending order. 
            # The code then prints out these 8 closest words so we can monitor how the embedding process 
            # is performing.
            
            if step % 1000 == 0:
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = reverse_dictionary[valid_examples[i]]
                    top_k = 8
                    nearest = (-sim[i, :]).argsort()[1: top_k + 1]
                    log_str = 'Nearest to %s:' %valid_word
                    for k in range(top_k):
                        close_word = reverse_dictionary[nearest[k]]
                        log_str = '%s %s,' % (log_str, close_word)
                    print(log_str)
                    
#--------------------------------------------------------------------------------------------------------------
        # Finally, after all the training iterations are finished, we can assign the final embeddings 
        # to a separate tensor for use later (most likely in some sort of other deep learning or
        # machine learning process)
        
        final_embeddings = normalized_embeddings.eval()

 To print out the words which are most similar to our validation words – we do this by calling the similarity operation we defined above and sorting the results

In [23]:
num_steps = 100
softmax_start_time = dt.datetime.now()
run(graph, num_steps=num_steps)
softmax_end_time = dt.datetime.now()
print("Softmax method took {} minutes to run 100 iterations".format((softmax_end_time-softmax_start_time).total_seconds()))


Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Average loss at step 0 is 9.373554229736328
Nearest to are: dawn, pursuing, legitimacy, appear, million, scales, countryside, isaac,
Nearest to more: coastal, peters, once, conscious, eternal, specialized, benjamin, absent,
Nearest to they: usually, layout, session, nationals, nfl, detail, contents, comet,
Nearest to often: dollars, williams, mazda, ep, joyce, react, manner, secret,
Nearest to most: isn, radios, dual, stream, formerly, recall, battle, rahman,
Nearest to such: judged, signed, shifting, memes, occupying, peer, javascript, uniforms,
Nearest to there: ryan, letter, brother, port, half, connection, rapper, ipv,
Nearest to so: permanently, resurrection, successes, decisions, mainframe, eliminate, notably, propose,
Nearest to not: investigations, operation, thomas, their, favor, reluctant, arab, hanging,
Nearest to seven: byron, gang, ruler, room, en, ethiopian, where, fascist,
Nearest to wa

In [24]:
with graph.as_default():

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

    nce_loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weights,
                       biases=nce_biases,
                       labels=train_context,
                       inputs=embed,
                       num_sampled=num_sampled,
                       num_classes=vocabulary_size))

    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(nce_loss)

    # Add variable initializer.
    init = tf.global_variables_initializer()

num_steps = 50000
nce_start_time = dt.datetime.now()
run(graph, num_steps)
nce_end_time = dt.datetime.now()
print("NCE method took {} minutes to run 50000 iterations".format((nce_end_time-nce_start_time).total_seconds()))

Initialized
Average loss at step 0 is 9.366310119628906
Nearest to are: genre, residence, composer, fate, hired, manga, associative, announcement,
Nearest to more: honors, anywhere, gathering, stay, bid, converting, produces, contributors,
Nearest to they: northeastern, photo, lt, caste, bow, endless, eugene, tremendous,
Nearest to often: gordon, amazon, click, apollo, scientists, green, consist, politics,
Nearest to most: evangelical, loose, interestingly, paramount, haydn, radios, offices, international,
Nearest to such: was, well, exodus, mechanisms, julius, whilst, sovereignty, site,
Nearest to there: kyrgyzstan, aerospace, arrow, monster, edited, imagery, spencer, tallest,
Nearest to so: utc, cruz, cited, ruins, neutrality, lesbian, walter, medicine,
Nearest to not: cayman, alpha, overthrow, melbourne, machinery, quad, nobel, chris,
Nearest to seven: muscles, diamond, holding, singers, confederacy, secure, stood, knowledge,
Nearest to war: sean, empirical, faster, paint, however, 

After every 10,000 iterations the code outputs the validation words and the words that the Word2Vec system deems are similar.  Below, you can see the improvement for some selected validation

After 1000 iterations:  
Nearest to seven: muscles, diamond, holding, singers, confederacy, secure, stood, knowledge,    
Nearest to over: cross, room, relies, lisbon, protected, jos, legends, cycles,

After 50000 iteration:  
Nearest to seven: eight, six, four, five, nine, three, zero, two,  
Nearest to over: into, room, cycles, through, at, only, cross, when,

We have learnt how to use the Word2Vec methodology to reduce large one-hot word vectors to much reduced word embedding vectors which preserve the context and meaning of the original words.  These word embedding vectors can then be used as a more efficient and effective input to deep learning techniques which aim to model natural language.

### de nada!